# Итоги выполнения домашнег задания к уроку 3

In [197]:
import numpy as np
import matplotlib.pyplot as plt

In [198]:
# данные взяла из задачи с урока 3
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика

In [199]:
# учитывая разброс показателей x в предложенном массиве провела стандартизацию 
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])

In [200]:
#функция предсказания вероятности 
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

### Задание 1 .Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where). С учетом озвученного на вебинаре задания, исключила так же 1.

С учетом озвученного на вебинаре задания, исключила так же 1.

Модифицирую функцию потерь calc_logloss таким образом, чтобы массив предсказанных значений y (y_pred) не содержал 0 и 1. Для этого все 0 в массиве будут приравниваться к 0,01, все единицы - к 0,99). Применяю функцию clip к данному входному массиву функции.

In [201]:
#функция потерь
def calc_logloss(y, y_pred):
    y_pred = np.clip(y_pred, 0.01, 0.99) # добавлена модификация значений в y_pred 
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

### Задание 2. Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным. Покажите влияние гиперпараметров на ошибку алгоритма (оптимально здесь использовать критерий остановки обучения). 

(при решении данной задачи применяла функцию calc_logloss с модификация значений предсказаний (без 0 и 1) y_pred из Задания 1)

In [202]:
# функция подбора коэффициентов w, которую буду тестировать
def eval_LR_model(X, y, alpha, diff):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    i = 0 # счетчик количества пройденных итераций
    och = 1 # переменная, определяющая разницу между err на текущем и предыдущем шаге/ итерации
    err_list = []
    i_list = []
    w_list = []
    while och >= diff:
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        err_list.append(err)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        w_list.append(w)
        i = i+1
        i_list.append(i)
        if len(err_list)> 2:
            och = err_list[-2]-err_list[-1]
    print(i_list[-2], w_list[-2], err_list[-2]) 
    print(i_list[-1], w_list[-1], err_list[-1])
           
    return w

Установила ограничение на изменение log_loss, которое задается пользователем. Посмотрела, как меняется количество операций для достижение заданного значения изменения при увеличении и уменьшении alpha.

Пример 1. 

In [203]:
w = eval_LR_model(X_st, y, alpha = 1e-2, diff = 0.000001)

57508 [-8.4568781  -1.24011996 -1.48573323  7.26901084] 0.2854870638172157
57509 [-8.4568781  -1.24011996 -1.48573323  7.26901084] 0.2854860638295222


In [204]:
y_pred = sigmoid(np.dot(X_st, w))
print(y_pred)
print(y)
calc_logloss(y, y_pred)

[0.2743669  0.1700346  0.98648447 0.21772387 0.71205276 0.34261467
 0.99609251 0.10050152 0.30589585 0.97137141]
[0 0 1 0 1 0 1 0 1 1]


0.28548506385975225

При ораничении изменения потерь значением 0,000001 при alpha 1e-2 на итерации 57509 подобраны значения w, при которых logloss снизилась относительно значения из модели из урока 3 (0,4993). Отосительно примера из урока снижение оибки достигнуто за счет увеличени количества итераций с 2000 до 57509. При данны параметрах подобраны w не позволяющие классицировать предсказанные y_pred аналогично y (2 значения y_pred классифицируются отличными от y при границе сравнения в 0,5).

Пример 2. 

In [205]:
w = eval_LR_model(X_st, y, alpha = 1e-3, diff = 0.000001)

68212 [-2.01943875 -0.9731686   0.84006604  2.78374195] 0.4333722258511486
68213 [-2.01943875 -0.9731686   0.84006604  2.78374195] 0.4333712258577682


In [206]:
y_pred = sigmoid(np.dot(X_st, w))
print(y_pred)
print(y)
calc_logloss(y, y_pred)

[0.26273811 0.33508103 0.77063562 0.29764466 0.81022293 0.61328034
 0.91493889 0.16463689 0.24630659 0.83821679]
[0 0 1 0 1 0 1 0 1 1]


0.43337022587415996

При ораничении изменения потерь значением 0,000001 при alpha 1e-3 на итерации 68213 подобраны значения w, , при которых log_loss выше, чем в Примере 1 и и лишь немного снизилась относительно задачи из урока 3 (0,4993). Т.о. параметры данного примера (alpha = 1e-1, 68213 итераций) не подходит для решения задачи по минимизации Log_loss.

Пример 3.

In [207]:
w = eval_LR_model(X_st, y, alpha = 1e-1, diff = 0.000001)

54958 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.14872080898236
54959 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.1487198090008968


In [208]:
y_pred = sigmoid(np.dot(X_st, w))
print(y_pred)
print(y)
calc_logloss(y, y_pred)

[0.34488811 0.02666186 0.99999977 0.10721192 0.89228513 0.0963408
 0.99999999 0.03221869 0.52432875 0.99999085]
[0 0 1 0 1 0 1 0 1 1]


0.14871880904455187

На данном примере подобраны параметры (alpha = 1e-1, количество итераций 54959), позволяющие минимизировать logloss до 0,1487, при этом w подбираются такие, что y_pred соответствует у при классификации полученных значений y_pred в 1 класс с вероятностью более 0,5.  
Данный пример является оптимальным.

### Задание 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred_proba).

In [209]:
def calc_pred_proba (X, w):
    y_pred_proba = sigmoid(np.dot(X, w))
    return y_pred_proba

In [210]:
calc_pred_proba (X_st, w)

array([0.34488811, 0.02666186, 0.99999977, 0.10721192, 0.89228513,
       0.0963408 , 0.99999999, 0.03221869, 0.52432875, 0.99999085])

### Задание 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred).

При решении в функцию в качестве входного параметра добавила границу вероятности, при которой объект относится к 1 или 0 классу (val_p). Если вероятность больше этой границы - объект относится к 1 классу. Параметр задается пользователем.

In [211]:
def calc_pred (X, w, val_p): 
    y_pred_proba = sigmoid(np.dot(X, w))
    y_pred = []
    for i in range(len(y_pred_proba)):
        if y_pred_proba[i] > val_p:
            y_pred.append(1)
        else:
            y_pred.append(0)           
    return y_pred

In [212]:
calc_pred_proba (X_st, w)

array([0.34488811, 0.02666186, 0.99999977, 0.10721192, 0.89228513,
       0.0963408 , 0.99999999, 0.03221869, 0.52432875, 0.99999085])

In [213]:
calc_pred (X_st, w, val_p = 0.9)

[0, 0, 1, 0, 0, 0, 1, 0, 0, 1]

### Задание 5. Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.

Для выполнения задания буду использовать код и модель, которая применялась на уроке.

In [214]:
def calc_logloss1(y, y_pred):
    err = np.mean(- y * np.log(y_pred) - (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [215]:
def eval_LR_model_1(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w1 = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w1) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss1(y, y_pred)
        w1 -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w1

In [216]:
w1 = eval_LR_model_1(X_st, y, 2000, 0.01)

200 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5687278067099053
400 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5476800278385013
600 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5369841864977791
800 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5295551503580798
1000 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5233765331724747
1200 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.517849884671324
1400 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5127677838840146
1600 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.5080360394424424
1800 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.503597783659264
2000 [-27.29523293  -2.83689291  -7.16656175  22.47024986] 0.4994117173889703


Расчет Accuracy (вариант 1)

In [217]:
# функция расчета показателя accuracy  
def AC (y, y_pred):
    accuracy = sum([1 for i in range(len(y)) if y_pred[i]==y[i]])/ len(y_pred)
    return accuracy

In [218]:
y_pred5 = calc_pred (X_st, w1, val_p = 0.5)
print(f'y_pred: {y_pred5}')
print(f'y: {y}')
print ( f'accurcy: {AC(y, y_pred5)}')

y_pred: [0, 0, 1, 0, 1, 1, 1, 0, 0, 1]
y: [0 0 1 0 1 0 1 0 1 1]
accurcy: 0.8


Accuracy по модели с урока = 0,8

Матрицу ошибок, precision, recall, accuracy и F1 реализовала в виде одной функции calc_quality (все метрики расчитываются с применением матрицы ошибок: 

In [219]:
# функция построения матрицы ошибок, на вход одаются истинные значения и значения, предсказанные моделью.

def calc_quality (y, y_pred):
    TP = 0 # правильно предсказано отнесение к 1 классу (количество)
    FP = 0 # неправильно предсказано отнесения к 1 классу (количество)
    FN = 0 # неправильно предсказано отнесения к 0 классу
    TN = 0 # правильно предсказано отнесение к 0 классу
    for i in range(len(y)):
        if y_pred[i]==y[i] and y_pred[i]==1:
            TP += 1
        elif y_pred[i]==y[i] and y_pred[i]==0:
            TN += 1
        elif y_pred[i]!=y[i] and y_pred[i]==1:
            FP += 1
        else:
            FN += 1
    
    print ( f'Матрица ошибок:{[[TP, FP], [FN, TN]]}')
    
    precision = TP/ (TP + FP)
    print ( f'precision: {precision}')
    
    recall = TP/ (TP + FN)
    print ( f'recall: {recall}')
    
    accuracy = (TP + TN)/len(y_pred)
    print ( f'accuracy: {accuracy}')
    
    F1 = 2 * precision * recall / (precision + recall) 
    print ( f'F1: {F1}')

Расчитала матрицу ошибок, precision, accuracy, recall и F1 по данным и модели с урока

In [220]:
y_pred5 = calc_pred (X_st, w5, val_p = 0.5)
print(f'y_pred: {y_pred5}')
print(f'y: {y}')
calc_quality(y, y_pred5)

y_pred: [0, 0, 1, 0, 1, 1, 1, 0, 0, 1]
y: [0 0 1 0 1 0 1 0 1 1]
Матрица ошибок:[[4, 1], [1, 4]]
precision: 0.8
recall: 0.8
accuracy: 0.8
F1: 0.8000000000000002


Метрики модели с урока:
Матрица ошибок:[[4, 1], [1, 4]] ;
precision: 0.8 ; 
recall: 0.8 ;
accuracy: 0.8 ;
F1: 0.8000000000000002

Дополнительно посчитала показаели модели, получившейся оптимальной в задании 2.

In [223]:
y_pred = calc_pred (X_st, w, val_p = 0.5)
print(f'y_pred: {y_pred}')
print(f'y: {y}')
calc_quality(y, y_pred)

y_pred: [0, 0, 1, 0, 1, 0, 1, 0, 1, 1]
y: [0 0 1 0 1 0 1 0 1 1]
Матрица ошибок:[[5, 0], [0, 5]]
precision: 1.0
recall: 1.0
accuracy: 1.0
F1: 1.0
